In [ ]:
# Requirement
!pip install openai -q
!pip install langchain -q
!pip install chromadb -q
!pip install tiktoken -q
!pip install unstructured[local-inference] -q
!pip install pypdf
!pip install gradio -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#nb the default google colab runtime loads PIL 8.4.0 and it won't work with unstructured document loader (see above dependency). 
import PIL
print(PIL.__version__)

9.5.0


In [ ]:
!pip uninstall Pillow
!pip install --upgrade Pillow
print(PIL.__version__)

Found existing installation: Pillow 9.5.0
Uninstalling Pillow-9.5.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/PIL/*
    /usr/local/lib/python3.10/dist-packages/Pillow-9.5.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libXau-154567c4.so.6.0.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libbrotlicommon-92722cb2.so.1
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libbrotlidec-db4b3db6.so.1.0.9
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libfreetype-cb9caf6f.so.6.19.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libharfbuzz-3543f599.so.0.60710.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libjpeg-f2134fdd.so.62.3.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/liblcms2-12745711.so.2.0.15
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/liblzma-95592ee6.so.5.4.2
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libopenjp2-78c47f58.so.2.5.0
    /usr/local/lib/python3.10/d

8.4.0


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain


import os
os.environ["OPENAI_API_KEY"] = ""
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,model_name="gpt-4")

In [ ]:
# Data Ingestion
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
#below directory loader not working with pdf file for some reason, so using PDFMinerLoader temporarily
#pdf_loader = DirectoryLoader('/content/Reports', glob="**/*.pdf")
txt_loader = DirectoryLoader('/content/Reports', glob="**/*.txt")
word_loader = DirectoryLoader('/content/Reports', glob="**/*.docx")

pdf_loader = PyPDFLoader("/content/Reports/HAI_AI-Index-Report_2023.pdf")

loaders = [pdf_loader, txt_loader, word_loader]
#loaders = [txt_loader, word_loader]
#loaders = [pdf_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Total number of documents: 388


In [ ]:
# Chunk and Embeddings
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

In [ ]:
from langchain.memory import ConversationBufferMemory

# Initialise Langchain - Conversation Retrieval Chain
#qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)

memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')
qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(temperature=0), 
    vectorstore.as_retriever(), 
    memory=memory,
    get_chat_history=lambda h : h,
    return_source_documents=True)


In [ ]:
#it won't read the pdf correctly it seems, also it doesn't seem to support chat history as a list any more as it fails on the second question.
# Front end web app
import json
from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.llms import OpenAI
from langchain.chains import ConversationChain


import gradio as gr
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    #chat_history = ()
    context = ChatMessageHistory()
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')
    #memory.chat_memory.add_user_message("hi!")
    #memory.chat_memory.add_ai_message("whats up?")
    #print(memory.load_memory_variables({}))
    #memory = ConversationBufferMemory(return_messages=True)
    #memory.chat_memory.add_user_message("hi!")
    #memory.chat_memory.add_ai_message("whats up?")
    #print(memory.load_memory_variables({}))

    #llm = OpenAI(temperature=0)
    #conversation = ConversationChain(
    #llm=llm, 
    #verbose=True, 
    #memory=ConversationBufferMemory()
#)
    #conversation.predict(input="Hi there!")

    #conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

    def dict_to_list(dict_input):
      return list(dict_input)

    def list_to_dict(list_input):
      return dict(list_input)
    
    def respond(user_message, chat_history):

        
        # Get response from QA chain
        response = qa({"question": user_message, "chat_history": chat_history})
        # Append user message and response to chat history
        chat_history.append((user_message, response["answer"]))
        return gr.update(value=""), chat_history
    msg.submit(respond, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
